In [ ]:
from indexer import build_faiss_index
from retriever import StudentRetriever
from obtain_mark import load_skills, generate_prompt, call_llm, compute_global_mark

/Users/albadiegovelarde/Library/CloudStorage/OneDrive-Repsol/Proyectos/student_marks/students_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Cargando google/flan-t5-large para generar summary...


Device set to use cpu


In [ ]:
YAML_DIR = "./student_reports"
INDEX_FILE = "students_info.index"
META_FILE = "students_metadata.npy"

In [3]:
LLM_MODEL_NAME = "google/flan-t5-large"

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import os 

print(f"Cargando {LLM_MODEL_NAME} para generar summary_short...")
tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL_NAME)
llm_model = AutoModelForSeq2SeqLM.from_pretrained(LLM_MODEL_NAME)
text_gen = pipeline(
    "text2text-generation",
    model=llm_model,
    tokenizer=tokenizer,
    max_new_tokens=150,
    device=0 if os.environ.get("CUDA_VISIBLE_DEVICES") else -1
)

Cargando google/flan-t5-large para generar summary_short...


Device set to use cpu


In [5]:
build_faiss_index(YAML_DIR, INDEX_FILE, META_FILE)

Cargando archivos YAML de alumnos...
Creando chunks y generando summaries con LLM...
Total de chunks: 11
Calculando embeddings con sentence-transformers...


ERROR:tornado.general:SEND Error: Host unreachable


Índice guardado en students_info.index
Metadatos guardados en students_metadata.npy


In [ ]:
STUDENTS_IDS = ["student_1", "student_2"]  # Lista de ids de alumnos
SKILLS_YAML = "/content/drive/MyDrive/Colab Notebooks/competences.yaml"

In [ ]:
skills = load_skills(SKILLS_YAML)
# Inicializar retriever
retriever = StudentRetriever()

# Diccionario final con todas las notas
student_grades = {}

for student_id in STUDENTS_IDS:
    print(student_id)
    student_grades[student_id] = {}
    for skill_name, skill_data in skills.items():
        print(skill_name)
        description = skill_data.get("description", "")
        instructions = skill_data.get("prompt_instructions", "")

        # Retrieval de chunks relevantes
        chunks = retriever.retrieve(description, student_id=student_id, top_k=TOP_K)

        if not chunks:
            print(f"No se encontraron chunks para student {student_id} y criterio '{description}'")
            continue

        # Generar prompt
        prompt = generate_prompt(instructions, chunks, description, skill_name)

        # Llamar al LLM
        llm_output = call_llm(prompt)

        # Guardar resultado
        student_grades[student_id][skill_name] = llm_output
    
    # Cálculo nota global ponderada
        global_mark = compute_global_mark(student_grades[student_id], skills)
        student_grades[student_id]["global_mark"] = global_mark

# Imprimir resultados
for student_id, grades in student_grades.items():
    print(f"\nAlumno: {student_id}")
    for skill_desc, result in grades.items():
        print(f"Criterio: {skill_desc}")
        print(f"Resultado LLM: {result}\n")
